# Estimate Sell Price Targets
Estimate the sell price targets of a stock trade
Should take into account:
- fees of trading, both fixed and variable
- costs with forex if existing, in the case of the currency of market being diferent of the currency of the brokerage 

<div class="admonition note">
  <p class="admonition-title">About the code for plots and tables</p>
  <p>
      The code for the plots and some tables is hidden on purpose.
      This is to avoid repeating code. Go to the repo to find the code.
  </p>
</div>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from bokeh.layouts import row
from bokeh.models import (ColumnDataSource, CrosshairTool, HoverTool, Label,
                          NumeralTickFormatter, PrintfTickFormatter, Span,
                          DataTable, TableColumn)
from bokeh.plotting import figure, show, output_notebook
from IPython.display import display_markdown

output_notebook()

Loading BokehJS ...

## Trade Details

In [2]:
# number of shares bought in this specific trade
trade_num_shares = 11
# price currency of the country/stock exchange where the stock is traded ($)
trade_price = 94.20
# currency exchange price used in the trade (EUR/USD)
trade_fx_price = 1.0697
# trade variable fees if existing, set by broker
# currency of the brokerage account (€)
trade_var_fee = 0.25/100 #0.25%
# trade fixed fees if existing, set by broker
# currency of the brokerage account (€)
trade_fixed_fee = 1

### Calculate trade value
in both currencies

In [3]:
trade_total_value_usd = trade_num_shares*trade_price
trade_total_value_eur = (trade_num_shares*trade_price)/trade_fx_price
trade_price_eur = trade_price/trade_fx_price

In [4]:
pd.DataFrame.from_dict({
    'exchange currency (USD)': (f'\${trade_total_value_usd:,.2f}', trade_num_shares, f'\${trade_price:,.2f}'),
    'account currency (EUR)': (f'{trade_total_value_eur:,.2f}€', trade_num_shares, f'{trade_price_eur:,.2f}€'),
}, orient='index', columns=['trade value', 'shares', 'trade price'])

,trade value,shares,trade price
exchange currency (USD),"\$1,036.20",11,\$94.20
account currency (EUR),968.68€,11,88.06€


### Calculate costs
in base/brokerage account currency

In [5]:
trade_var_fee_value = (trade_var_fee*trade_total_value_eur)*-1 # should be in negative value
trade_fixed_fee_value = trade_fixed_fee*-1
trade_total_cost_eur = trade_var_fee_value+trade_fixed_fee_value
trade_unit_cost_eur = trade_total_cost_eur/trade_num_shares

In [6]:
pd.DataFrame.from_dict(
    {
        "variable fee": (
            f"{trade_var_fee_value:,.2f}€",
            f"{trade_var_fee_value/trade_num_shares:,.2f}€",
        ),
        "fixed fee": (
            f"{trade_fixed_fee_value:,.2f}€",
            f"{trade_fixed_fee_value/trade_num_shares:,.2f}€",
        ),
        " total cost": (
            f"{trade_total_cost_eur:,.2f}€",
            f"{trade_unit_cost_eur:,.2f}€",
        ),
    },
    orient="index",
    columns=["cost value", "cost\/share"],
)

,cost value,cost\/share
variable fee,-2.42€,-0.22€
fixed fee,-1.00€,-0.09€
total cost,-3.42€,-0.31€


## Calculate sell target prices
- selling all shares in one single trade
- use the most recent broker fees and fx rate

In [7]:
# Download currency exchange stuff
eur_usd = yf.Ticker('EURUSD=X')

In [8]:
pd.DataFrame.from_dict(
    {
        "previous Close": f'{eur_usd.info["previousClose"]:,.4f}€',
        "52 Week Low": f'{eur_usd.info["fiftyTwoWeekLow"]:,.4f}€',
        "52 Week High": f'{eur_usd.info["fiftyTwoWeekHigh"]:,.4f}€',
        "50 Day Average": f'{eur_usd.info["fiftyDayAverage"]:,.4f}€',
        "200 Day Average": f'{eur_usd.info["twoHundredDayAverage"]:,.4f}€',
    },
    orient="index",
    columns=["EUR/USD"],
)

,EUR/USD
previous Close,1.0949€
52 Week Low,0.9540€
52 Week High,1.1276€
50 Day Average,1.0927€
200 Day Average,1.0730€


In [9]:
# fx value at the time of sale base on the last available price
curr_fx_price = eur_usd.info['previousClose']

sell_var_fee = 0.25/100 # set by broker 0.25%
sell_fixed_fee = 2 # set by broker, currency of the brokerage account (€), value by trade

# selling all shares
sell_num_shares = trade_num_shares

### Calculate Break Even Price (BEP)


In [10]:
# calculate total trade value in euros before costs (GROSS) using latest FX costs
gross_sell_bep_value_eur = (sell_num_shares*trade_price)/trade_fx_price
gross_sell_bep_price_eur = gross_sell_bep_value_eur/sell_num_shares

# should be in negative values
sell_var_fee_value = (sell_var_fee*gross_sell_bep_value_eur)*-1
sell_fixed_fee_value = sell_fixed_fee*-1
sell_total_cost_value_eur = sell_var_fee_value+sell_fixed_fee_value+trade_total_cost_eur

# calculate after costs (NET)
net_sell_bep_value_eur = gross_sell_bep_value_eur-sell_total_cost_value_eur
net_sell_bep_price_eur = net_sell_bep_value_eur/sell_num_shares

In [11]:
pd.DataFrame.from_dict(
    {"EUR/USD": f"{curr_fx_price:,.4f}€"},
    orient="index",
    columns=["current forex exchange"],
)

,current forex exchange
EUR/USD,1.0949€


In [12]:
pd.DataFrame.from_dict(
    {
        "gross BEP": (
            f"{gross_sell_bep_value_eur:,.2f}€",
            f"\${gross_sell_bep_value_eur*curr_fx_price:,.2f}",
            f"{gross_sell_bep_price_eur:,.2f}€",
            f"\${gross_sell_bep_price_eur*curr_fx_price:,.2f}",
        ),
        "variable sell fee": (
            f"{sell_var_fee_value:,.2f}€",
            "",
            f"{sell_var_fee_value/sell_num_shares:,.2f}€",
            "",
        ),
        "fixed sell fee": (
            f"{sell_fixed_fee_value:,.2f}€",
            "",
            f"{sell_fixed_fee_value/sell_num_shares:,.2f}€",
            "",
        ),
        "total cost (buy/sell)": (
            f"{sell_total_cost_value_eur:,.2f}€",
            "",
            f"{sell_total_cost_value_eur/sell_num_shares:,.2f}€",
            "",
        ),
        "net BEP": (
            f"{net_sell_bep_value_eur:,.2f}€ ",
            f"\${net_sell_bep_value_eur*curr_fx_price:,.2f}",
            f"{net_sell_bep_price_eur:,.2f}€",
            f"\${net_sell_bep_price_eur*curr_fx_price:,.2f}",
        ),
    },
    orient="index",
    columns=["total value", "total value", "unit price", "unit price"],
)

,total value,total value,unit price,unit price
gross BEP,968.68€,"\$1,060.64",88.06€,\$96.42
variable sell fee,-2.42€,,-0.22€,
fixed sell fee,-2.00€,,-0.18€,
total cost (buy/sell),-7.84€,,-0.71€,
net BEP,976.53€,"\$1,069.23",88.78€,\$97.20


### Calculate loss of selling at buy price

In [13]:
pct_loss_buy_price = ((trade_price / trade_fx_price) - net_sell_bep_price_eur) / (
    trade_price / trade_fx_price
)

### Estimate % target

In [14]:
target_grow = 0.15

target_gross_price = gross_sell_bep_price_eur * (1 + target_grow)
target_gross_value = target_gross_price * sell_num_shares

# add the fixed and variable transaction fees
target_net_value = (
    (target_gross_value + sell_fixed_fee) / (1 - sell_var_fee)
) - trade_total_cost_eur
target_net_price = target_net_value / sell_num_shares

In [15]:
display_markdown(f"""
| BEP price | grow target | net target price |
|---:|---:|---:|
| {net_sell_bep_price_eur:,.2f}€ | {target_grow:.1%} | {target_net_price:,.2f}€ |
| \${net_sell_bep_price_eur*curr_fx_price:,.2f} | {target_grow:.1%} | \${target_net_price*curr_fx_price:,.2f} |
""", raw=True)


| BEP price | grow target | net target price |
|---:|---:|---:|
| 88.78€ | 15.0% | 102.02€ |
| \$97.20 | 15.0% | \$111.70 |


#### Estimate % targets

In [16]:
# grow percentage targets between 0% (BEP) and 25%
TARGETS = np.arange(0.0, 0.51, 0.01)

def calc_target(target_grow, bep_price, num_shares, fixed_fee, var_fee, buy_costs):
    gross_value = bep_price * (1 + target_grow) * num_shares

    # add the fixed&variable transaction fees
    net_value = (gross_value + fixed_fee) / (1 - var_fee) - buy_costs
    return net_value / num_shares


output_acc = []
for t in TARGETS:
    out = calc_target(
        t,
        gross_sell_bep_price_eur,
        sell_num_shares,
        sell_fixed_fee,
        sell_var_fee,
        trade_total_cost_eur,
    )
    output_acc.append(out)
output_exc = list(map(lambda x: x * curr_fx_price, output_acc))

In [17]:
source = ColumnDataSource(
    data=dict(targets=TARGETS, prices_exc=output_exc, prices_acc=output_acc)
)


fig = figure(
    title="Target Prices",
    x_axis_label="grow target %",
    y_axis_label="stock price target EUR",
    toolbar_location="above",
    sizing_mode="stretch_width",
    height=600,
)

line = fig.line(
    x="targets",
    y="prices_acc",
    legend_label="target",
    line_width=1,
    source=source,
    name="target_line",
    color="black",
)

fig.circle(
    0.0,
    net_sell_bep_price_eur,  # , *curr_fx_price,
    legend_label="BEP",
    color="green",
    size=5,
)
fig.add_layout(
    Span(
        location=net_sell_bep_price_eur,  # *curr_fx_price,
        dimension="width",
        line_color="green",
        line_width=1,
    )
)

fig.circle(
    pct_loss_buy_price,
    trade_price / trade_fx_price,
    legend_label="entry price",
    color="orange",
    size=5,
)
fig.add_layout(
    Span(
        location=trade_price / trade_fx_price,
        dimension="width",
        line_color="orange",
        line_width=1,
    )
)

fig.add_layout(
    Label(
        x=2.5,
        y=net_sell_bep_price_eur,
        x_units="screen",
        text=f"{net_sell_bep_price_eur:.2f}€",
        border_line_width=2,
        border_line_color="green",
        background_fill_color="green",
        text_font_size="12px",
        text_color="white",
    )
)

fig.add_layout(
    Label(
        x=2.5,
        y=trade_price / trade_fx_price,
        x_units="screen",
        text=f"{trade_price/trade_fx_price:.2f}€",
        border_line_width=2,
        border_line_color="orange",
        background_fill_color="orange",
        text_font_size="12px",
        text_color="white",
        text_baseline="top",
    )
)

# fig.toolbar.logo = None
fig.toolbar.autohide = True
fig.legend.location = "top_left"
fig.legend.click_policy = "hide"
# fig.yaxis[0].formatter = NumeralTickFormatter(format=' 0.00')
fig.yaxis[0].formatter = PrintfTickFormatter(format="%0.2f €")
fig.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")

fig.add_tools(CrosshairTool(dimensions="height"))
tooltips = [
    ("pct", "$snap_x{0.0%}"),
    ("price", "@prices_exc{0.00}€ - @prices_acc{$0.00}"),
]
fig.add_tools(HoverTool(tooltips=tooltips, mode="vline", renderers=[line]))


columns = [
    TableColumn(field="targets", title="target Pct"),
    TableColumn(field="prices_acc", title="Price EUR"),
    TableColumn(field="prices_exc", title="Price USD"),
]
data_table = DataTable(
    source=source,
    columns=columns,
    width=300,
    height=600,
    selectable=False,
    index_position=None,
)

show(row(fig, data_table, sizing_mode="scale_width"))